<a href="https://colab.research.google.com/github/SabihaKhanum/PythonFastAPI/blob/main/hackathon_bugsomething.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

so i am basically, fetching data via Alpha Vantage.
creating the csv
pre-process the data
LSTM

In [2]:
pip install alpha_vantage

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from datetime import datetime, timedelta
import math


In [4]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries

api_key = 'OCXCT0FV6PJNPL9P'
ts = TimeSeries(key=api_key, output_format='pandas')
#add company variable here
data, meta_data = ts.get_intraday(symbol='AAPL', interval='1min', outputsize='full')
stock_data = data.iloc[::-1]
stock_data.to_csv('stock_data.csv', index_label='time')
list(stock_data)

['1. open', '2. high', '3. low', '4. close', '5. volume']

**Open**: This column represents the opening price of the stock on the current trading day.
**High**: This column represents the highest price of the stock reached during the trading day.
**Low**: This column represents the lowest price of the stock reached during the trading day.
**Last**: This column represents the price of the stock at the last transaction of the trading day.
**Close**: This column represents the closing price of the stock on the current trading day.

In [5]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

stock_data = pd.read_csv('stock_data.csv', index_col='time')

# Normalize stock data
scaler = MinMaxScaler()
stock_data_scaled = scaler.fit_transform(stock_data[['1. open', '2. high', '3. low', '4. close', '5. volume']])
close_scaler = MinMaxScaler()
close_scaler.fit(stock_data[['4. close']])
# Split data
train_data = stock_data_scaled[:int(0.8*len(stock_data_scaled))] #training set (used to train mode)
test_data = stock_data_scaled[int(0.8*len(stock_data_scaled)):] #testing set

why use minmax scalar?
basically scales the entire dataset between 0 and 1

what else can i use instead of minmax?
standard scalar, normalisation etc.

In [6]:
import torch
import torch.nn as nn

# Define LSTM model
class StockPredictionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (hidden, cell) = self.lstm(x)
        hidden = hidden[-1]
        output = self.linear(hidden)
        return output

# Initialize model, loss, and optimizer
input_size = 5  # Number of features
hidden_size = 64
output_size = 1  # Predicting closing price
model = StockPredictionLSTM(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train model
epochs = 100
for epoch in range(epochs):
    inputs = torch.from_numpy(train_data[:-1]).unsqueeze(0).float()
    targets = torch.from_numpy(train_data[1:, 3]).unsqueeze(0).float()

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 11499])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


used GPT4 cause my god is it tough.

In [7]:
test_inputs = torch.from_numpy(test_data[:-1]).unsqueeze(0).float()
test_predictions = model(test_inputs).detach().numpy()
print(test_predictions)

# Inverse transform predictions using the closing price scaler
test_predictions_rescaled = close_scaler.inverse_transform(test_predictions.reshape(-1, 1))
print(test_predictions_rescaled)

[[0.50818205]]
[[172.69543]]


In [8]:
recent_data = stock_data_scaled[-1]
#again, add the variable.
num_future_days = 5

# Predict future prices
future_predictions = []
for _ in range(num_future_days):
    prediction = model(torch.from_numpy(recent_data).unsqueeze(0).float()).detach().numpy()
    future_predictions.append(prediction)
    recent_data = np.roll(recent_data, -1)
    recent_data[-1] = prediction

# Inverse transform predictions
future_predictions_rescaled = close_scaler.inverse_transform(future_predictions) #rescaled basically is removing the normalisation
print(future_predictions_rescaled)

[[164.35524514]
 [164.11732945]
 [164.09978531]
 [163.68845268]
 [163.48131705]]


<ipython-input-8-5248ebaaa145>:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  recent_data[-1] = prediction


In [9]:
current_price = close_scaler.inverse_transform(stock_data_scaled[-1, 3].reshape(-1, 1))

# Create trading signals based on predictions
trading_signals = []
for prediction in future_predictions_rescaled:
    if prediction > current_price * 1.01:  # 1% above the current price
        trading_signals.append('Buy')
    elif prediction < current_price * 0.99:  # 1% below the current price
        trading_signals.append('Sell')
    else:
        trading_signals.append('Hold')
print(trading_signals)

['Sell', 'Sell', 'Sell', 'Sell', 'Sell']
